# WZE-UAV Image Classification using Deep Learning

In [1]:
import os
import gc
import glob
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold

In [2]:
import torch
import torchvision.transforms as T
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
from mlxtend.plotting import plot_confusion_matrix

In [3]:
import wze_uav.data_loader as data_loader
import wze_uav.models as models
from wze_uav.engine import *
from wze_uav.utils2 import *
#from wze_uav.log_writer import create_writer
from wze_uav.datasplit import *
from efficientnet import model_effnet #for custom effnet with n_channels input
import wandb

#### Get PyTorch version

In [4]:
print(f"torch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")

torch version: 1.13.1+cu116
torchvision version: 0.14.1+cu116


#### Preparing device agnostic code 

In [5]:
# ensure device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# get index of currently selected device
print(f"Index of current divice: {torch.cuda.current_device()}")
# get number of GPUs available
print(f"Number of GPUs available: {torch.cuda.device_count()}")
# get the name of the device
print(f"GPU Model: {torch.cuda.get_device_name(0)}")

cuda
Index of current divice: 0
Number of GPUs available: 1
GPU Model: Quadro RTX 8000


#### Login to Weights & Biases to track results

In [6]:
# IMPORTANT: The proxy needs to be set in anaconda!
wandb.login()
#wandb.init(settings=wandb.Settings(start_method="thread"))
wandb.init(project='wze-uav', entity='simon-ecke')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: simon-ecke. Use `wandb login --relogin` to force relogin


#### Ensure reproducibility 

In [7]:
# for more information, see also: https://pytorch.org/docs/stable/notes/randomness.html

# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)
    # seed for numpy
    np.random.seed(seed)

set_seeds(42) 

# Set to true -> might speed up the process but should be set to False if reproducible results are desired
torch.backends.cudnn.benchmark = False


#### Define file directory

In [8]:
#####################################################################################
# 3 channel input (r-g-b)
#data_path = r"D:\Drohnendaten\10_WZE-UAV\Auswertung_findatree\Datasplit\ROI\rgb"

# 4 channel input (r-g-b-nir)
data_path = r"D:\Drohnendaten\10_WZE-UAV\Auswertung_findatree\Datasplit\ROI\rgb-nir"

# 5 channel input (r-g-b-re-nir)
#data_path = r"D:\Drohnendaten\10_WZE-UAV\Auswertung_findatree\Datasplit\ROI\rgb-re-nir"
#####################################################################################

#### Get all file paths

In [9]:
fn_list = os.listdir(data_path)
path_list = []
# Iterate over all datafiles
for year in fn_list:
    year_dir = f'{data_path}\\{year}'
    for filename in os.listdir(year_dir):
        path = f'{year_dir}\\{filename}'
        path_list.append(path)

In [29]:
import findatree_roi.exporter as exporter
from collections import defaultdict
import hashlib

def get_unique_treeID_v2(path_list: str):
    enr = {}
    bnr = {}
    sat = {}
    all_id = {}
    count = 0
    for fn in tqdm(path_list, desc="Creating unique tree IDs..."):
        # load hdf5 data from path
        rois, params_rois, data, params_data = exporter.load_rois_from_hdf5_v2(fn,
                                                                               load_sets=["images_masked"])
       
        # fetch terrestrial features (such as labels and species etc.)
        tnr = fn.rsplit("\\",1)[1].split("_",1)[0].split("r",1)[1]
        crowns_human = data['crowns_human'] #get crown info with all terrestrial features
        enr = crowns_human['features']['terrestrial']['enr']
        sat = crowns_human['features']['terrestrial']['sat'] 
        bnr = crowns_human['features']['terrestrial']['bnr'] 
        year = fn.rsplit("\\", 2)[1]
        for i in range(len(enr)):
            all_id[count] = {'img_id': count, 'tnr': tnr, 'enr': enr[i], 'sat': sat[i], 'bnr': bnr[i], 'year': year}
            count += 1
            
      # First, group the data by the values of tnr, enr, sat, and bnr
        grouped_data = defaultdict(list)
        for idx, item in all_id.items():
            key = (item['tnr'], item['enr'], item['sat'], item['bnr'])
            grouped_data[key].append((idx, item))
        
        # Then, create the final dictionary and calculate the hash IDs
        final_dict = {}
        for key, group in grouped_data.items():
            hash_id = hashlib.sha1(str(key).encode('utf-8')).hexdigest()[:8]
            final_dict[hash_id] = [(idx, item['img_id'], item['year']) for idx, item in group]
    
    # Update the new_dict dictionary comprehension to include the year
    new_dict = {idx: (final_dict[hash_id], final_dict[hash_id][idx][1]) for hash_id in final_dict for idx in range(len(final_dict[hash_id]))}
    hashID_dict = dict(new_dict.items())

    for hashID, index_year_list in final_dict.items():
        for index, img_id, year in index_year_list:
            hashID_dict[index] = (hashID, year, img_id)
        
    hashID_dict = dict(sorted(hashID_dict.items(), key=lambda x: x[0]))
    
    return hashID_dict

#### Create unique hash IDs for every individual tree

In [30]:
hashID_dict = get_unique_treeID_v2(path_list)

Creating unique tree IDs...:   0%|          | 0/647 [00:00<?, ?it/s]

In [31]:
hashID_dict

{0: ('a25bf9b5', '2020', 0),
 1: ('d92527a4', '2020', 1),
 2: ('48f235ea', '2020', 2),
 3: ('f75528a1', '2020', 3),
 4: ('069109ae', '2020', 4),
 5: ('82b5fb2a', '2020', 5),
 6: ('02485a5d', '2020', 6),
 7: ('80300fb3', '2020', 7),
 8: ('1355ff14', '2020', 8),
 9: ('479276ef', '2020', 9),
 10: ('a7d93e3d', '2020', 10),
 11: ('5ca73ebf', '2020', 11),
 12: ('a7c9e7d9', '2020', 12),
 13: ('b033525a', '2020', 13),
 14: ('cee38d34', '2020', 14),
 15: ('8d9169c3', '2020', 15),
 16: ('06454d29', '2020', 16),
 17: ('69b3cc07', '2020', 17),
 18: ('2afcdd76', '2020', 18),
 19: ('2bba0a03', '2020', 19),
 20: ('1cb5b645', '2020', 20),
 21: ('91812420', '2020', 21),
 22: ('2c601a30', '2020', 22),
 23: ('d49b1d15', '2020', 23),
 24: ('0b467e03', '2020', 24),
 25: ('9b36403c', '2020', 25),
 26: ('516c8f64', '2020', 26),
 27: ('77f9b0d1', '2020', 27),
 28: ('4ceaf6d8', '2020', 28),
 29: ('45bbb3dc', '2020', 29),
 30: ('f983435d', '2020', 30),
 31: ('2580f27b', '2020', 31),
 32: ('ba9e6410', '2020', 32

#### Import all imagery, labels and other features from hdf5 files

In [32]:
image_set, label_set, species_set, kkl_set, bk_set, hash_id = data_loader.hdf5_to_img_label(path_list,
                                                                                               hashID_dict,
                                                                                               load_sets=["images_masked"])

Processing hdf5 datasets:   0%|          | 0/647 [00:00<?, ?it/s]

#### Convert nbv to classes

In [33]:
label_set = nbv_to_sst_3classes(label_set)

In [66]:
from sklearn.model_selection import GroupShuffleSplit


def data_split_v2(image_set: np.array, label_set: np.array, hash_id: np.array, species_set: np.array, test_size=0.1667, random_state=42):
    
    # Get the unique values of hashID and their corresponding indices
    #unique_values, unique_indices = np.unique(hash_id[:,0], return_index=True)
    # Get the corresponding label values for the unique hashID values
    #unique_labels = label_set[unique_indices]
    #num_unique = len(unique_values)
    #num_indices = unique_indices
    #print(f"There are {num_unique} unique values within hash_id.")
    #print(f"There are {num_indices} unique indices within hash_id.")
    
    # get image IDs for each unique hash ID and year
    image_ids_2020 = hash_id[hash_id[:, 1] == '2020'][:, 0]
    image_ids_2021 = hash_id[hash_id[:, 1] == '2021'][:, 0]
    image_ids_2022 = hash_id[hash_id[:, 1] == '2022'][:, 0]
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    # Initialize the GroupShuffleSplit function with the desired test_size and random_state
    for sub_index, test_index in gss.split(X=hash_id[:, 0], y=label_set, groups=hash_id[:, 0]):
        # get the hash ids for the training and validation sets
        sub_ids = hash_id[:, 0][sub_index]
        test_ids = hash_id[:, 0][test_index]
    
    # Split the data into train and test sets based on both hashID and label while ensuring a stratified split
    #sub_ids, test_ids = next(gss.split(X=hash_id[:,0], y=label_set, groups=hash_id[:,0]))
    print(sub_ids)
    print(len(sub_ids))
    print(test_ids)
    print(len(test_ids))
    
    # create boolean arrays indicating which indices belong to train or test sets
    # 1. combined subset of training and validation data
    sub_indices = np.isin(hash_id[:, 2], sub_ids)
    num_true = np.count_nonzero(sub_indices)
    num_false = np.count_nonzero(~sub_indices)
    print("Number of True in sub_indices:", num_true)
    print("Number of False in sub_indices:", num_false)
    
    # 2. test dataset
    #np.random.seed(seed) # set the seed value for randomly choosing an image of one of the years
    test_indices = np.zeros_like(sub_indices)  # initialize to all False
    for hash_id_test in test_ids:
        # select one image ID randomly from either 2020 or 2021 or 2022 for each unique hash ID in the test set
        year = np.random.choice(['2020', '2021', '2022'])
        image_ids = hash_id[(hash_id[:, 0] == hash_id_test) & (hash_id[:, 1] == year), 0]
        
        # mark the index corresponding to the selected image ID and hash ID as True in the test indices array
        test_indices[(hash_id[:, 0] == hash_id_test) & (hash_id[:, 1] == year) & (np.isin(hash_id[:, 0], image_ids))] = True
  
    num_true = np.count_nonzero(test_indices)
    num_false = np.count_nonzero(~test_indices)
    print("Number of True in test_indices:", num_true)
    print("Number of False in test_indices:", num_false)
    
    # Reshape boolean arrays to match shape of image_set and label_set
    sub_indices = sub_indices.reshape(-1, 1)
    test_indices = test_indices.reshape(-1, 1)
    
    # Select images and labels for train_val and test sets
    sub_image_set = image_set[sub_indices[:, 0]]
    sub_label_set = label_set[sub_indices[:, 0]]
    sub_hash_id = hash_id[sub_indices[:, 0]]
    sub_species_set = species_set[sub_indices[:, 0]]
    
    test_image_set = image_set[test_indices[:, 0]]
    test_label_set = label_set[test_indices[:, 0]]
    test_hash_id = hash_id[test_indices[:, 0]]
    test_species_set = species_set[test_indices[:, 0]]
    
    sub_label_set = sub_label_set.reshape(-1, 1)
    test_label_set = test_label_set.reshape(-1, 1)
    sub_species_set = sub_species_set.reshape(-1, 1)
    test_species_set = test_species_set.reshape(-1, 1)

    print("Check shapes:\n")
    print(f"Images sub dataset: {sub_image_set.shape}")
    print(f"Labels sub dataset: {sub_label_set.shape}\n")
    print(f"Images test dataset: {test_image_set.shape}")
    print(f"Labels test dataset: {test_label_set.shape}\n")
    print('-'*50)
    print (f"Check if the split was stratified: (random_state={random_state})")
    print(f"Healthy trees in sub dataset: {np.count_nonzero(sub_label_set == 0)}")
    print(f"Stressed trees in sub dataset: {np.count_nonzero(sub_label_set == 1)}")
    print(f"Dead trees in sub dataset: {np.count_nonzero(sub_label_set == 2)}")
    print(f"Healthy trees in test dataset: {np.count_nonzero(test_label_set == 0)}")
    print(f"Stressed trees in test dataset: {np.count_nonzero(test_label_set == 1)}")
    print(f"Dead trees in test dataset: {np.count_nonzero(test_label_set == 2)}")
    print(f"Ratio health trees in test dataset: {np.count_nonzero(test_label_set == 0)/np.count_nonzero(sub_label_set == 0)}")
    print(f"Ratio stressed trees in test dataset: {np.count_nonzero(test_label_set == 1)/np.count_nonzero(sub_label_set == 1)}")
    print(f"Ratio dead trees in test dataset: {np.count_nonzero(test_label_set == 2)/np.count_nonzero(sub_label_set == 2)}")
    
    del image_set, label_set, hash_id, species_set
    return sub_image_set, sub_label_set, sub_hash_id, sub_species_set, test_image_set, test_label_set, test_hash_id, test_species_set

#### Split data into a sub set and a test dataset

In [67]:
sub_image_set, sub_label_set, sub_hash_id, sub_species_set, test_image_set, test_label_set, test_hash_id, test_species_set = data_split_v2(image_set, label_set, hash_id, species_set, test_size=0.1667, random_state=42)


['a25bf9b5' 'd92527a4' '48f235ea' ... '26ae74cb' 'cc39eb77' '7302ad23']
15659
['a7d93e3d' 'b033525a' '69b3cc07' ... '1efb8a5c' 'cbdb390e' '804903c3']
3146
Number of True in sub_indices: 0
Number of False in sub_indices: 18805
Number of True in test_indices: 2107
Number of False in test_indices: 16698
Check shapes:

Images sub dataset: (0, 250, 250, 4)
Labels sub dataset: (0, 1)

Images test dataset: (2107, 250, 250, 4)
Labels test dataset: (2107, 1)

--------------------------------------------------
Check if the split was stratified: (random_state=42)
Healthy trees in sub dataset: 0
Stressed trees in sub dataset: 0
Dead trees in sub dataset: 0
Healthy trees in test dataset: 1244
Stressed trees in test dataset: 835
Dead trees in test dataset: 28


ZeroDivisionError: division by zero

#### Check if any hash ID is in both sub and test dataset

In [38]:
hash_set = set(sub_hash_id[:,0].flatten())
test_hash_set = set(test_hash_id[:,0].flatten())
intersection = hash_set.intersection(test_hash_set)
if intersection:
    print(f"Hash_id values in both train and test sets: {intersection}")
else:
    print("There are no hash_id values in both train and test datasets. The datasplit was successful")

There are no hash_id values in both train and test datasets. The datasplit was successful


#### Check feature distribution of the Test dataset

In [19]:
def count_occurrences(data, value):
    count = 0
    for item in data:
        if item == value:
            count += 1
    return count

print("Test dataset")
print(f"Test data healthy trees: {count_occurrences(test_label_set, 0)}")
print(f"Test data stressed trees: {count_occurrences(test_label_set, 1)}")
print(f"Test data dead trees: {count_occurrences(test_label_set, 2)}")
print(f"Test data pine trees: {count_occurrences(test_species_set, 134)}")
print(f"Test data spruces: {count_occurrences(test_species_set, 118)}")
print("-"*50)

print("Remaining dataset")
print(f"Remaining data healthy trees: {count_occurrences(sub_label_set, 0)}")
print(f"Remaining data stressed trees: {count_occurrences(sub_label_set, 1)}")
print(f"Remaining data dead trees: {count_occurrences(sub_label_set, 2)}")
print(f"Remaining data pine trees: {count_occurrences(sub_species_set, 134)}")
print(f"Remaining data spruces: {count_occurrences(sub_species_set, 118)}")
print("-"*50)

Test dataset
Test data healthy trees: 612
Test data stressed trees: 429
Test data dead trees: 15
Test data pine trees: 278
Test data spruces: 388
--------------------------------------------------
Remaining dataset
Remaining data healthy trees: 9232
Remaining data stressed trees: 6166
Remaining data dead trees: 261
Remaining data pine trees: 3960
Remaining data spruces: 5833
--------------------------------------------------


In [20]:
# train transform with augmentation. 
transform_train = transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip(p=0.5), transforms.RandomVerticalFlip(p=0.5),
                                      transforms.RandomRotation(degrees=[0,360])])

# test and val dataset transform without augmentation. 
transform = transforms.Compose([transforms.ToTensor()])

# class names need to fit the customDataset class used e.g. 3 classes -> use CustomDataset3Classes
#class_names = ['healthy', 'slightly_stressed', 'moderately_stressed', 'highly_stressed', 'dead']
#class_names = ['healthy', 'moderately_stressed', 'highly_stressed', 'dead']
class_names = ['healthy', 'stressed', 'dead']

# set seeds
g = torch.Generator()
g.manual_seed(42)
NUM_WORKERS=3 # should be changed, depending on the system used
batch_size=32

#### Define variables and parameters

In [21]:
# 1. Define number of epochs
epochs = 50
n_bands = sub_image_set[0].shape[2] # get number of bands

# 2. Define model
num_classes = len(class_names)
unfreeze = True # all layer weights get updated
dropout_rate = 0.45 #define dropout rate
model_name = "EffNet_b7_RGB-NIR_3classes_lr0005"

# 3. Define loss, optimizer and learning rate scheduler
lr = 0.0005 # define learning rate
min_lr = 1e-6 # minimum learning rate threshold
gamma = 0.85 # how fast the learning rate decreases per epoch (low number=faster decrease)
patience = 10

# 4. Create target folder name were to save the tensorboard event files
experiment_name = 'RGB-NIR_3classes_v3_lr0005'
extra = "new_val_set_new_classes_lr0005"

#----------------------------------------------------------------------
#torch.cuda.empty_cache()
print(f"Memory allocated: {torch.cuda.memory_allocated()} bytes") 



Memory allocated: 0 bytes


In [22]:
# login to weights & biases to track metrics
#!set http_proxy = http://www-proxy.bayern.de:80
#!set https_proxy = http://www-proxy.bayern.de:80
#os.environ['http_proxy'] = 'http://www-proxy.bayern.de:80'
#os.environ['https_proxy'] = 'http://www-proxy.bayern.de:80'


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [24]:
#import requests
#r = requests.get('https://www.google.com')
#print(r.status_code)

#### Run k-Fold cross-validation on EfficientNet

In [25]:
%%time

# Set the random seeds
set_seeds(42)

# Define the number of folds
num_folds = 5

# Create a KFold object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

unique_values = np.unique(sub_hash_id[:,0]) # now there are less unique values left because we seperated the test dataset
num_unique = len(unique_values)
print(f"There are {num_unique} unique values within hash_id.\n")

# Loop over each fold
for fold, (train_ids, val_ids) in enumerate(kf.split(unique_values)):
    torch.cuda.empty_cache()
    
    # Initialize a new wandb run for this fold
    wandb.init(project='wze-uav', name=f"fold_{fold + 1}_{extra}")
    
    # 1. Split data into train and validation set
    # Get the training and testing data for this fold
    # Use np.isin() to create boolean arrays indicating which indices belong to train or test sets
    train_indices = np.isin(sub_hash_id[:,0], unique_values[train_ids])
    
    val_indices = np.zeros_like(train_indices)  # initialize to all False
    for hash_id_val in unique_values[val_ids]:
        # select one image ID randomly from either 2020 or 2021 or 2022 for each unique hash ID in the test set
        year = np.random.choice(['2020', '2021', '2022'])
        image_ids = sub_hash_id[(sub_hash_id[:,0] == hash_id_val) & (sub_hash_id[:,1] == year), 0]
    
        # mark the index corresponding to the selected image ID and hash ID as True in the test indices array
        val_indices[(sub_hash_id[:,0] == hash_id_val) & (sub_hash_id[:,1] == year) & (np.isin(sub_hash_id[:,0], image_ids))] = True 
    
    # Reshape boolean arrays to match shape of image_set and label_set
    train_indices = train_indices.reshape(-1, 1)
    val_indices = val_indices.reshape(-1, 1)
    
    # Select images and labels for train and validation sets
    train_image_set = sub_image_set[train_indices[:, 0]]
    train_label_set = sub_label_set[train_indices[:, 0]]
    train_hash_id = sub_hash_id[train_indices[:, 0]][:,0]
    train_species_set = sub_species_set[train_indices[:, 0]]
    val_image_set = sub_image_set[val_indices[:, 0]]
    val_label_set = sub_label_set[val_indices[:, 0]]
    val_hash_id = sub_hash_id[val_indices[:, 0]][:,0]
    val_species_set = sub_species_set[val_indices[:, 0]]
    # reshape 
    train_label_set = train_label_set.reshape(-1, 1)
    val_label_set = val_label_set.reshape(-1, 1)
    train_species_set = train_species_set.reshape(-1, 1)
    val_species_set = val_species_set.reshape(-1, 1)

         
    print("Check shapes:\n")
    print(f"Images train dataset: {train_image_set.shape}")
    print(f"Labels train dataset: {train_label_set.shape}\n")
    
    print(f"Images validation dataset: {val_image_set.shape}")
    print(f"Labels validation dataset: {val_label_set.shape}\n")
    print('-'*50)
    print (f"Check if the split was stratified: (random_state=42)")
    print(f"Healthy trees in train dataset: {np.count_nonzero(train_label_set == 0)}")
    print(f"Stressed trees in train dataset: {np.count_nonzero(train_label_set == 1)}")
    print(f"Dead trees in train dataset: {np.count_nonzero(train_label_set == 2)}")
    print(f"Healthy trees in validation dataset: {np.count_nonzero(val_label_set == 0)}")
    print(f"Stressed trees in validation dataset: {np.count_nonzero(val_label_set == 1)}")
    print(f"Dead trees in validation dataset: {np.count_nonzero(val_label_set == 2)}")
    print(f"Ratio health trees in validation dataset: {np.count_nonzero(val_label_set == 0)/np.count_nonzero(sub_label_set == 0)}")
    print(f"Ratio stressed trees in validation dataset: {np.count_nonzero(val_label_set == 1)/np.count_nonzero(sub_label_set == 1)}")
    print(f"Ratio dead trees in validation dataset: {np.count_nonzero(val_label_set == 2)/np.count_nonzero(sub_label_set == 2)}")
    print("-"*50)
   
    # 2. Create train and validation dataset. (choose custom dataset loader with 3 - 5 classes)
    print(f"\nCreating datasets for fold: {fold + 1}\n")
    train_dataset = data_loader.CustomDataset(data=train_image_set, labels=train_label_set, class_names=class_names, species = train_species_set,
                                                         transform=transform_train)
    
    val_dataset = data_loader.CustomDataset(data=val_image_set, labels=val_label_set, class_names=class_names,
                                                       species = val_species_set, transform=transform)
   
    # 3. Create train and validation dataloader
    # create sampler for oversampling of the minority classes
    sampler = data_loader.data_sampler(dataset=train_dataset, class_names=class_names)
    print(f"Creating dataloaders for fold: {fold +1}\n")
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, persistent_workers=True, pin_memory=True, num_workers=NUM_WORKERS, generator=g,
                              sampler=sampler, shuffle=False, drop_last=True) # shuffle false because of the sampler

    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, persistent_workers=True, pin_memory=True, num_workers=NUM_WORKERS, shuffle=False,
                             drop_last=True)
    
    model = model_effnet.EfficientNet.from_pretrained('efficientnet-b7', in_channels=n_bands, num_classes=num_classes, dropout_rate=dropout_rate)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
    #lr_scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    lr_scheduler = ExponentialLR(optimizer, gamma=gamma)

    fold += 1
    print(f"\n[INFO] Fold number: {fold}")
    print(f"[INFO] Number of epochs: {epochs}")
    print(f"[INFO] Batch_size: {batch_size}")
    print(f"[INFO] Number of bands: {n_bands}")
    print(f"[INFO] Dropout rate: {dropout_rate}")
    print(f"[INFO] Gamma learning rate: {gamma}")
    print(f"[INFO] Memory allocated: {torch.cuda.memory_allocated()} bytes")
    # 4. Train model with k fold dataloaders and track experiments
    
    if fold == 1:
        fold1_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
       
    elif fold == 2:
        fold2_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
    elif fold == 3:
        fold3_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
    elif fold == 4:
        fold4_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
    else:
        fold5_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
    
    del train_indices, val_indices, train_image_set, train_label_set, train_hash_id, train_species_set, val_image_set, val_label_set, val_hash_id, val_species_set,
    train_dataset, val_dataset, sampler, train_dataloader, val_dataloader, model, loss_fn, optimizer, lr_scheduler
    
    #finish the wandb run
    wandb.finish()
    print("Deleting variables and emptying cache")
    gc.collect()
    torch.cuda.empty_cache()
    print(f"Memory allocated: {torch.cuda.memory_allocated()} bytes")
    print("-"*50 + "\n")
    


There are 5905 unique values within hash_id.



Check shapes:

Images train dataset: (12564, 250, 250, 4)
Labels train dataset: (12564, 1)

Images validation dataset: (1043, 250, 250, 4)
Labels validation dataset: (1043, 1)

--------------------------------------------------
Check if the split was stratified: (random_state=42)
Healthy trees in train dataset: 7401
Stressed trees in train dataset: 4959
Dead trees in train dataset: 204
Healthy trees in validation dataset: 618
Stressed trees in validation dataset: 404
Dead trees in validation dataset: 21
Ratio health trees in validation dataset: 0.06694107452339688
Ratio stressed trees in validation dataset: 0.06552059682127798
Ratio dead trees in validation dataset: 0.08045977011494253
--------------------------------------------------

Creating datasets for fold: 1

Creating dataloaders for fold: 1

Loaded pretrained weights for efficientnet-b7

[INFO] Fold number: 1
[INFO] Number of epochs: 50
[INFO] Batch_size: 32
[INFO] Number of bands: 4
[INFO] Dropout rate: 0.45
[INFO] Gamma lear

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 
Learning rate: 0.0005
Train loss: 0.7025 | Train precision: 0.6242 | Train recall: 0.6065 | Train f1score: 0.6149 | Train acc: 0.6051 | Train kappa: 0.3371 
Val loss: 0.7661 | Val precision: 0.7308 | Val recall: 0.4288 | Val f1score: 0.4085 | Val acc: 0.4990 | Val kappa: 0.1464 

Epoch: 2 
Learning rate: 0.000425
Train loss: 0.6399 | Train precision: 0.7021 | Train recall: 0.6909 | Train f1score: 0.6962 | Train acc: 0.6509 | Train kappa: 0.4193 
Val loss: 0.6773 | Val precision: 0.5917 | Val recall: 0.7071 | Val f1score: 0.6306 | Val acc: 0.6123 | Val kappa: 0.3409 

Epoch: 3 
Learning rate: 0.00036124999999999997
Train loss: 0.6270 | Train precision: 0.7194 | Train recall: 0.7100 | Train f1score: 0.7145 | Train acc: 0.6622 | Train kappa: 0.4497 
Val loss: 0.6931 | Val precision: 0.6874 | Val recall: 0.7415 | Val f1score: 0.7040 | Val acc: 0.6309 | Val kappa: 0.3706 

Epoch: 4 
Learning rate: 0.0003070625
Train loss: 0.6211 | Train precision: 0.7296 | Train recall: 0.7256 | T

Epoch: 28 
Learning rate: 6.212708809110104e-06
Train loss: 0.5321 | Train precision: 0.7997 | Train recall: 0.7980 | Train f1score: 0.7987 | Train acc: 0.7256 | Train kappa: 0.5580 
Val loss: 0.6179 | Val precision: 0.7091 | Val recall: 0.7386 | Val f1score: 0.7223 | Val acc: 0.6670 | Val kappa: 0.4047 

Epoch: 29 
Learning rate: 5.280802487743588e-06
Train loss: 0.5366 | Train precision: 0.7909 | Train recall: 0.7907 | Train f1score: 0.7907 | Train acc: 0.7201 | Train kappa: 0.5524 
Val loss: 0.6160 | Val precision: 0.7056 | Val recall: 0.7345 | Val f1score: 0.7188 | Val acc: 0.6631 | Val kappa: 0.3965 

Early stopping after epoch 29


...

Check shapes:

Images train dataset: (12522, 250, 250, 4)
Labels train dataset: (12522, 1)

Images validation dataset: (1040, 250, 250, 4)
Labels validation dataset: (1040, 1)

--------------------------------------------------
Check if the split was stratified: (random_state=42)
Healthy trees in train dataset: 7369
Stressed trees in train dataset: 4954
Dead trees in train dataset: 199
Healthy trees in validation dataset: 618
Stressed trees in validation dataset: 401
Dead trees in validation dataset: 21
Ratio health trees in validation dataset: 0.06694107452339688
Ratio stressed trees in validation dataset: 0.06503405773597146
Ratio dead trees in validation dataset: 0.08045977011494253
--------------------------------------------------

Creating datasets for fold: 5

Creating dataloaders for fold: 5

Loaded pretrained weights for efficientnet-b7

[INFO] Fold number: 5
[INFO] Number of epochs: 50
[INFO] Batch_size: 32
[INFO] Number of bands: 4
[INFO] Dropout rate: 0.45
[INFO] Gamma lear

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 
Learning rate: 0.0005
Train loss: 0.7112 | Train precision: 0.6057 | Train recall: 0.5945 | Train f1score: 0.5999 | Train acc: 0.6061 | Train kappa: 0.3239 
Val loss: 0.7612 | Val precision: 0.5240 | Val recall: 0.4400 | Val f1score: 0.4447 | Val acc: 0.6221 | Val kappa: 0.2799 

Epoch: 2 
Learning rate: 0.000425
Train loss: 0.6424 | Train precision: 0.7120 | Train recall: 0.7032 | Train f1score: 0.7074 | Train acc: 0.6510 | Train kappa: 0.4210 
Val loss: 0.6589 | Val precision: 0.6981 | Val recall: 0.5005 | Val f1score: 0.5406 | Val acc: 0.6436 | Val kappa: 0.2960 

Epoch: 3 
Learning rate: 0.00036124999999999997
Train loss: 0.6337 | Train precision: 0.7184 | Train recall: 0.7131 | Train f1score: 0.7154 | Train acc: 0.6586 | Train kappa: 0.4363 
Val loss: 0.6316 | Val precision: 0.7481 | Val recall: 0.6672 | Val f1score: 0.7014 | Val acc: 0.6777 | Val kappa: 0.4085 

Epoch: 4 
Learning rate: 0.0003070625
Train loss: 0.6155 | Train precision: 0.7250 | Train recall: 0.7296 | T

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
fold,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,█▇▆▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁
train_f1_score,▁▅▅▆▆▆▇▇▇▇▇▇▇██▇████
train_loss,█▅▅▄▃▃▃▃▃▃▂▂▂▁▂▂▂▁▁▁
val_f1_score,▁▃▇▆▆██▇▆██▇▇█▇▇▇▇▇█
val_loss,█▃▂▄▆▁▂▂▅▁▃▃▃▂▂▃▃▂▂▃
epoch,19
fold,5
learning_rate,2e-05
train_f1_score,0.78299


Deleting variables and emptying cache
Memory allocated: 1055400960 bytes
--------------------------------------------------

CPU times: total: 5h 41min 24s
Wall time: 11h 49min 49s


In [ ]:
# Create a graph to visualize the training and validation loss for each fold
wandb.run.summary["graph"] = wandb.plot.line_series(
    xs="epoch",
    ys=["train_loss", "val_loss"],
    group="fold",
    xaxis="Epoch",
    yaxis="Loss",
    title="Training and Validation Loss by Fold",
)

# Create a graph to visualize the training and validation f1 score for each fold
wandb.run.summary["graph"] = wandb.plot.line_series(
    xs="epoch",
    ys=["train_f1_score", "val_f1_score"],
    group="fold",
    xaxis="Epoch",
    yaxis="Loss",
    title="Training and Validation F1-Score by Fold",
)

In [ ]:
# create test dataset
test_dataset = data_loader.CustomTestDataset(
    data = test_image_set,
    labels = test_label_set,
    class_names=class_names, 
    species = test_species_set,
    kkl = None,
    transform=transform
)

# create test dataloader
test_dataloader = DataLoader(test_dataset,
                             batch_size=batch_size,
                             persistent_workers=True,
                             pin_memory=True,
                             num_workers=NUM_WORKERS,
                             shuffle=False,
                             drop_last=True)

In [ ]:
# Setup the best model filepath
best_model_path = r"C:\Users\lwfeckesim\01_PyTorch\wze-uav\wze-uav-master\effnet_b0\01_18_epochs.pth"

# Instantiate a new instance of EffNetB0 (to load the saved state_dict() to)
unfreeze=True
best_model = models.create_effnetb0(output_shape=num_classes, unfreeze=unfreeze, dropout_rate=dropout_rate, device=device)

# Load the saved best model state_dict()
best_model.load_state_dict(torch.load(best_model_path))

In [ ]:
def make_predictions(model: torch.nn.Module, 
                     test_dataloader: torch.utils.data.DataLoader,
                     device: torch.device):
    # 1. Make predictions with trained model
    y_preds = []
    y_labels = []
    species_list = []
    test_loss, test_precision, test_recall, test_f1_score, test_acc = 0, 0, 0, 0, 0
    count = 0
    model.eval()
    with torch.inference_mode():
        for X, y, species in tqdm(test_dataloader, desc="Making predictions"):
            # Send data and targets to target device
            X, y = X.to(device), y.to(device)
            # Do the forward pass
            y_logit = model(X)
            # Turn predictions from logits -> prediction probabilities -> predictions labels
            y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1)
            # Put predictions on CPU for evaluation
            y_preds.append(y_pred.cpu())
            y_labels.append(y.cpu())
            species_list.append(species)
            
            #other metrics
            test_acc += ((y_pred == y).sum().item()/len(y_pred))
            y_pred_class = y_pred.detach().cpu().numpy() 
            y_class = y.detach().cpu().numpy()
            labels = np.array([0])
            test_precision += precision_score(y_class, y_pred_class, average='macro', zero_division=0, labels=[0,1,2])
            test_recall += recall_score(y_class, y_pred_class, average='macro', zero_division=0, labels=[0,1,2])
            #test_f1_score += f1_score(y_class, y_pred_class, average='macro', zero_division=1, labels=labels)
            
            #if count >= 1:
            #    y_set = torch.cat((y_set, y))
            #    count = count + 1
            #else:
            #    y_set = y
            #    count = count + 1
            
    test_loss = test_loss / len(test_dataloader)
    test_precision = test_precision / len(test_dataloader)
    test_recall = test_recall / len(test_dataloader)
    test_f1_score = test_f1_score / len(test_dataloader)
    #test_kappa = test_kappa / len(dataloader)
    test_acc = test_acc / len(test_dataloader)
    # Concatenate list of predictions into a tensor
    y_pred_tensor = torch.cat(y_preds)
    y_labels_tensor = torch.cat(y_labels)
    test_f1_score = f1_score(y_labels_tensor.detach().cpu().numpy(), y_pred_tensor.cpu().numpy(), average='macro', zero_division=1, labels=[0,1,2])
    
    # Print classification report
    y_true = y_labels_tensor.detach().cpu().numpy()
    report = classification_report(y_true, y_pred_tensor.cpu().numpy(), target_names=class_names)
    print(report)
    
    return y_pred_tensor, y_labels_tensor, test_loss, test_recall, test_precision, test_f1_score, test_acc, y_logit, y_pred, y, y_preds

In [ ]:
# 2. Setup confusion matrix instance and compare predictions to targets
#from wze_uav.analysis import *
y_pred_tensor, y_labels_tensor, test_loss, test_recall, test_precision, test_f1_score, test_acc, y_logit, y_pred, y, y_preds = make_predictions(model=best_model,
                                 test_dataloader=test_dataloader, 
                                 device=device)

y_labels_tensor = y_labels_tensor.detach().cpu().numpy()
y_pred_tensor = y_pred_tensor.detach().cpu().numpy()

#confmat = ConfusionMatrix(num_classes=num_classes, task='multiclass')
#confmat_tensor = confmat(preds=y_pred_tensor,
#                         target=test_labels)
labels = np.array([0,1,2])
confmat = confusion_matrix(y_labels_tensor, y_pred_tensor, labels=labels)

# 3. Plot the confusion matrix
fig, ax = plot_confusion_matrix(
    conf_mat=confmat, # matplotlib likes working with NumPy 
    class_names=class_names, # turn the row and column labels into class names
    figsize=(10, 7)
);

print(f"Test loss: {test_loss}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Test F1score: {test_f1_score}")
#print(f"Test Kappa: {test_kappa}")
print(f"Test Accuracy: {test_acc}")
print(f"Test Logits: {y_logit}")
print(f"Test Predictions: {y_pred}")
print(f"Test Labels: {y}")

In [ ]:
len(y_preds)

In [ ]:
y_set.cpu()

In [ ]:
test_dataset.labels

In [ ]:
y_preds = []
y_labels = []
labels = np.array([0,1,2])
test_loss, test_precision, test_recall, test_f1_score, test_acc = 0, 0, 0, 0, 0
count = 0
model.eval()
with torch.inference_mode():
    for X, y in tqdm(test_dataloader, desc="Making predictions"):
        # Send data and targets to target device
        X, y = X.to(device), y.to(device)
        # Do the forward pass
        y_logit = model(X)
        # Turn predictions from logits -> prediction probabilities -> predictions labels
        y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1)
        # Put predictions on CPU for evaluation
        y_preds.append(y_pred.cpu())
        y_labels.append(y.cpu())
        
        #other metrics
        test_acc += ((y_pred == y).sum().item()/len(y_pred))
        y_pred_class = y_pred.detach().cpu().numpy() 
        y_class = y.detach().cpu().numpy()
        test_precision += precision_score(y_class, y_pred_class, average='macro', zero_division=1, labels=labels)
        test_recall += recall_score(y_class, y_pred_class, average='macro', zero_division=1, labels=labels)
        #test_f1_score += f1_score(y_class, y_pred_class, average='macro', zero_division=1, labels=labels)
        
        #if count >= 1:
        #    y_set = torch.cat((y_set, y))
        #    count = count + 1
        #else:
        #    y_set = y
        #    count = count + 1
        
test_loss = test_loss / len(test_dataloader)
test_precision = test_precision / len(test_dataloader)
test_recall = test_recall / len(test_dataloader)
#test_f1_score = test_f1_score / len(test_dataloader)
#test_kappa = test_kappa / len(dataloader)
test_acc = test_acc / len(test_dataloader)
# Concatenate list of predictions into a tensor
y_pred_tensor = torch.cat(y_preds)
test_f1_score = f1_score(y_set.detach().cpu().numpy(), y_pred_tensor.cpu().numpy(), average='macro', zero_division=0, labels=[0,1,2])

# Print classification report
y_true = y_set.detach().cpu().numpy()
report = classification_report(y_true, y_pred_tensor.cpu().numpy(), target_names=class_names)
print(report)

In [ ]:
test_f1_score

In [ ]:
make = (y_class == y_pred_class)
make

In [ ]:
torch.softmax(y_logit, dim=1).argmax(dim=1)

In [ ]:
y

In [ ]:
test = (y_pred == y).sum().item()/16

In [ ]:
test

In [ ]:
 y_pred_class = y_pred.detach().cpu().numpy() 

In [ ]:
y_pred_class